In [ ]:
import sys

sys.path.append('/workspace/current/loce')

from bg_randomized_loce.utils.loce_storage_helpers import pkl_todo_count
from bg_randomized_loce.utils.consts import _BG_DATASET_KEY, _MODEL_KEY, _DATASET_KEY, _CE_METHOD_KEY, DATA_BUILDERS
from typing import get_args
import itertools
from tqdm.notebook import tqdm
import pandas as pd

exclude_cats = {
    'pascal_voc': [21, 22, 23, 24, 25],
    'imagenets50': ['ignore', 1000],
}

results_dir = '/workspace/current/loce/results'
all_combis = list(
    itertools.product(get_args(_CE_METHOD_KEY), get_args(_DATASET_KEY), get_args(_BG_DATASET_KEY), (1, 4, 8, 32),
                      get_args(_MODEL_KEY)))
count_done, count_in_progress = 0, 0
results_list = []
for ce_method, dataset_key, bg_randomizer_key, num_bgs_per_ce, model_key in tqdm(all_combis):
    for category_id, category_name in DATA_BUILDERS[dataset_key][0]:
        if category_id in exclude_cats[dataset_key]: continue
        if bg_randomizer_key == 'vanilla' and num_bgs_per_ce > 1: continue

        num_todo = pkl_todo_count(results_dir, run='run1', ce_method=ce_method, bg_randomizer_key=bg_randomizer_key,
                               num_bgs_per_ce=num_bgs_per_ce, dataset_key=dataset_key, model_key=model_key,
                               category_id=category_id)

        results_list.append(
            dict(status='TODO' if num_todo > 0 else 'DONE', remaining=num_todo,
                 ce_method=ce_method, dataset_key=dataset_key, bg_randomizer_key=bg_randomizer_key,
                 num_bgs_per_ce=num_bgs_per_ce, model_key=model_key, category_id=category_id))
        #print(f"{status}\t {_file_glob}")

rdf = pd.DataFrame(results_list)

In [ ]:
# only the used ce_methods
frdf = rdf[rdf['ce_method'].isin(['net2vec_proper_bce', 'loce_proper_bce'])]
# only 1 bg per file for vanilla bg_randomizer_key
frdf = frdf[~((frdf['num_bgs_per_ce'] > 1) & (frdf['bg_randomizer_key'] == 'vanilla'))]
# just exclude those where none is remaining
frdf = frdf[frdf['remaining'] >= 0]

# print results
for ce_method in frdf['ce_method'].unique():
    for stat in frdf['status'].unique():
        print(stat, ce_method, len(frdf[(frdf['status'] == stat) & (frdf['ce_method'] == ce_method)].index) / len(
            frdf[frdf['ce_method'] == ce_method].index) * 100)

frdf[frdf['status'] == 'TODO']

In [ ]:
todo = frdf[frdf['status'] != 'DONE']
#for c in frdf.columns: print(c, todo[todo['num_bgs_per_ce'] < 16][c].unique())
todo[todo['num_bgs_per_ce'] == 1]

In [ ]:
todo = frdf[frdf['status'] != 'DONE']
#for c in frdf.columns: print(c, todo[todo['num_bgs_per_ce'] < 16][c].unique())
display(todo)